In [32]:
import matplotlib
matplotlib.use("Agg")
import scipy
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import linear_model
from copy import deepcopy
import statsmodels.api as sm

In [33]:
GROUP = 'Group'
RESULT = 'Result'
CONSTANT = 'Intercept_'
ITNAME = 'Item name'
alpha = 0.05
SKIPLINE = '\n'            #Yes, I wish I didn't have to do this but /n just won't work...

data = pd.read_excel("Dataset.xlsx")
data.insert(0, CONSTANT, 1)
lin_model = sm.OLS

C:\Users\Javid\AppData\Roaming\Python\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [34]:
# split into groups

group_labels = np.unique(data[GROUP])
groups = []
for l in group_labels:
    g = data.loc[data[GROUP]==l]
    groups.append(g)

In [35]:
n_sampsize, m_vars = np.shape(data.drop([RESULT,GROUP],axis = 1))
m_vars -= 1

k_models = len(groups)

df_t = n_sampsize - m_vars - 1
total_model = sm.OLS(data[RESULT],data.drop([RESULT,GROUP],axis = 1)).fit()
errors = data[RESULT]-total_model.predict(data.drop([RESULT,GROUP],axis = 1))
sst = np.sum(np.square(errors))

df_c = n_sampsize - m_vars - k_models
combined_models = []
ssc = 0
for g in groups:
    
    m = deepcopy(total_model)
    m.params[CONSTANT] = 0
    
    zeros_shape = (np.size(g[RESULT]),1)
    flat_data = g[RESULT] - m.predict(g.drop([RESULT,GROUP],axis = 1))
    #flat_model = lin_model().fit(np.zeros(zeros_shape),flat_data)
    #m.intercept_ = flat_model.intercept_
    #print(flat_model.intercept_ - np.mean(flat_data))
    m.params[CONSTANT] = np.mean(flat_data)
    
    combined_models.append(m)
    errors = g[RESULT]-m.predict(g.drop([RESULT,GROUP],axis=1))
    ssc += np.sum(np.square(errors))
    
df_p = n_sampsize - k_models*(m_vars + 1)
partial_models = []
ssp=0
for g in groups:
    m = sm.OLS(g[RESULT],g.drop([RESULT,GROUP],axis = 1)).fit()
    partial_models.append(m)
    errors = g[RESULT]-m.predict(g.drop([RESULT,GROUP],axis=1))
    ssp += np.sum(np.square(errors))

In [36]:
f_tot = ((sst-ssp)/(df_t-df_p))/(ssc/df_p)
dfs_tot = (df_t-df_p,df_p)

f_slopes = ((ssc-ssp)/(df_c-df_p))/(ssp/df_p)
dfs_slopes = (df_c-df_p,df_p)

f_elevations = ((sst-ssc)/(df_t-df_c))/(ssc/df_c)
dfs_elevations = (df_t-df_c,df_c)

def print_fstats():
    print('f total is:     ',f_tot, '\twith dfn/dfd: ',dfs_tot)
    print('f slopes is:    ',f_slopes, '\twith dfn/dfd: ',dfs_slopes)
    print('f elevations is:',f_elevations, '\twith dfn/dfd: ',dfs_elevations)

In [37]:
def get_text_stats(f,dfs,prob):
    if prob < 0.0001:
        pstr = 'P<0.0001'
    else:
        pstr = 'P=' + str(round(prob,4))
    
    s = 'F = '+ str(round(f,4)) +'. '
    s += 'DFn = ' + str(dfs[0]) + ', DFd = '+ str(dfs[1])
    s += SKIPLINE + pstr
    
    return s
    
def get_text_1(prob, item):
    if prob > 0.05:
        issig = ' are not significant.'
    else:
        issig = ' are significant.'
    
    if prob < 0.0001:
        prob = 'less than 0.01%'
        issig = ' are extremely significant.'
    else:
        prob = str(round(prob*100,2))+'%'
    
    s = 'If the overall ' + item
    s += ' were identical, there is a ' + prob
    s += ' chance of randomly choosing data points with ' + item
    s += ' this different. You can conclude that the differences between the ' + item + issig
    
    return  s

def get_text_2(slope):
    slope = str(round(slope,5))
    s = 'Since the slopes are not significantly different, it is possible to calculate one slope for all the data.'
    s += SKIPLINE + 'The pooled slope equals ' +slope+'.'
    
    return s

def generate_plain_lang(f_slopes,dfs_slopes,f_elevations,dfs_elevations):
    
    p_slopes = 1-scipy.stats.f.cdf(f_slopes,dfs_slopes[0],dfs_slopes[1])
    p_elevations = 1-scipy.stats.f.cdf(f_elevations,dfs_elevations[0],dfs_elevations[1])
    
    s = 'Are the slopes equal?' + SKIPLINE
    s += get_text_stats(f_slopes,dfs_slopes,p_slopes) + SKIPLINE*2
    s += get_text_1(p_slopes,'slopes')
    
    if p_slopes <= 0.05:
        return s
    
    s += SKIPLINE*2
    s += 'Are the elevations equal?' + SKIPLINE
    s += get_text_stats(f_elevations,dfs_elevations,p_elevations) + SKIPLINE*2
    s += get_text_1(p_elevations,'elevations')
    return s
# print(generate_plain_lang(f_slopes,dfs_slopes,f_elevations,dfs_elevations))

In [38]:
f_groups_slopes = []
dfs_groups_slopes = []


for i,g in enumerate(groups):
    n_g = np.size(g[RESULT])
    
    total_ss = np.sum(np.square(g[RESULT] - np.mean(g[RESULT])))
    residual_ss = np.sum(np.square(g[RESULT] - partial_models[i].predict(g.drop([RESULT,GROUP],axis=1))))
    reggression_ss = total_ss - residual_ss
    
    df_res = n_g - m_vars - 1
    df_reg = m_vars
    
    f = (reggression_ss/df_reg)/(residual_ss/df_res)
    dfs = (df_reg,df_res)
    
    f_groups_slopes.append(f)
    dfs_groups_slopes.append(dfs)

In [44]:
def generate_tabular_data():
    
    df = pd.DataFrame(columns= [ITNAME] + list(group_labels))
    i = 0
    df.loc[i,ITNAME] = 'Best-fit values'
    df.loc[i+1] = np.nan
    i += 2

    coef_names = list(data.drop([RESULT,GROUP],axis = 1).columns)
    for c_name in coef_names:
        df.loc[i,ITNAME] = c_name
        df.loc[i+1,ITNAME] = "Std. Error"
        df.loc[i+2,ITNAME] = "95% Confidence Interval"
        df.loc[i+3] = np.nan
        for j,m in enumerate(partial_models):
            col_name = group_labels[j]
            df.loc[i,col_name] = str(m.params[c_name])
            df.loc[i+1,col_name] = str(np.sqrt(m.cov_params().loc[c_name,c_name]))

            ci_df = m.conf_int(alpha)
            CI = [ci_df.loc[c_name,0],ci_df.loc[c_name,1]]

            df.loc[i+2,col_name] = '['+str(CI[0])+' , '+str(CI[1])+']'

        i+= 4

    df.loc[i,ITNAME] = 'Goodness of fit'
    df.loc[i+1,ITNAME] = 'R squared'
    df.loc[i+2,ITNAME] = 'Sy.x'
    df.loc[i+3] = np.nan

    for j,m in enumerate(partial_models):
        col_name = group_labels[j]
        df.loc[i+1,col_name] = str(m.rsquared)
        df.loc[i+2,col_name] = str(np.sqrt(m.mse_resid))
    i+= 4

    df.loc[i,ITNAME] = 'Is slope significantly non-zero?'
    df.loc[i+1,ITNAME] = 'F'
    df.loc[i+2,ITNAME] = 'DFn, DFd'
    df.loc[i+3,ITNAME] = 'P value'
    df.loc[i+4,ITNAME] = 'Deviation from zero?'

    for j,m in enumerate(partial_models):
        col_name = group_labels[j]

        fval = f_groups_slopes[j]
        dfs = dfs_groups_slopes[j]
        pval = 1-scipy.stats.f.cdf(fval,dfs[0],dfs[1])
        issig = 'Not significant'
        if pval <= 0.05:
            issig = 'Significant'

        if pval < 0.0001:
            pval = '<0.0001'
        else:
            pval = str(pval)

        df.loc[i+1,col_name] = str(fval)
        df.loc[i+2,col_name] = str(dfs)
        df.loc[i+3,col_name] = pval
        df.loc[i+4,col_name] = issig
        
    return df

In [45]:
tabular_results = generate_tabular_data()
plain_lang = generate_plain_lang(f_slopes,dfs_slopes,f_elevations,dfs_elevations)

In [ ]:
tabular_results.to_excel("tabular_results.xlsx", index=False)

In [ ]:
with open("plain_lang.txt", "a") as f:
  f.write(plain_lang)

print(plain_lang)

Are the slopes equal?
F = 0.6992. DFn = 6, DFd = 24
P=0.6529

If the overall slopes were identical, there is a 65.29% chance of randomly choosing data points with slopes this different. You can conclude that the differences between the slopes are not significant.

Are the elevations equal?
F = 2.9241. DFn = 3, DFd = 30
P=0.0499

If the overall elevations were identical, there is a 4.99% chance of randomly choosing data points with elevations this different. You can conclude that the differences between the elevations are significant.
